# Pandas Introduction

In this week's lecture we're going to deepen our investigation to how Python can be used to manipulate, clean, and query data by looking at the Pandas data tool kit.

Pandas was created by Wes McKinney in 2008, and is an open source project under a very permissive license. It has a strong community, with over one hundred software developers all committing code to help make it better. Pandas borrows many concepts from R and provides a consistent interface to numpy, the python core libraries, some plotting routines and a variety of python statistical tools.

## Resources for learning pandas

Pandas is an enormous library, and can be overwhelming in its scale.

### Stackoverflow

Stack Overflow is used broadly within the software development community to post questions about programming, programming languages, and programming toolkits. The Pandas community, in particular, uses it as their number one resource for helping new members. It's quite possible if you post a question to Stack Overflow, and tag it as being Pandas and Python related, that a core Pandas developer will actually respond to your question. Stack Overflow is also a great place to go to see what issues people are having and how they can be solved.

### Books

In 2012 Wes McKinney wrote the definitive Pandas reference book called _Python for Data Analysis_ and published by O'Reilly, and it has been updated since. Another good reference is _Learning the Pandas Library_ by Matt Harrison. If you just want to learn the basics of Pandas and want to do so quickly, it is well laid out can be had for a good price.

### Blogs

data science and machine learning are evolving rapidly. Marco Rodriguez and Tim Golden maintain a wonderful blog aggregator site called Planet Python. You can visit the webpage at planetpython.org. There's lots of regular Python data science contributors.

### Podcasts

Kyle Polich runs an excellent podcast called Data Skeptic. It has interviews with varied experts in the field as well as short educational lessons. Much of the word he describes is specific to machine learning methods.

## Summary

Of course this is just a fraction of the resources available! Indeed, these notebooks draw upon upon the work of others. I may add other resources later. Please share any useful resources you find in the slack workspace associated with this module.

# The `Series` Datastructure

The first Pandas concept we will examine is the _series_ data structure. By the end, you should be familiar with how to store and manipulate single dimensional indexed data in the Series object.

The series is one of the core data structures in pandas. It is a cross between a list and a dictionary. The items are all stored in an order and there are labels with which you can retrieve them. An easy way to visualize this is two columns of data. The first is the special index, a lot like keys in a dictionary. The second is your actual data. It's important to note that the data column has a label of its own and can be retrieved using the `.name` attribute. A dictionary does not have this attribute, but it is useful later when merging multiple columns of data.

Let's import pandas to get started

In [1]:
import pandas as pd

As you might expect, you can create a series by passing in a list of values. When you do this, Pandas automatically assigns an index starting with zero and sets the name of the series to None. Let's work on an example of this.

One of the easiest ways to create a series is to use an array-like object, like a list. 

In [2]:
# Here I'll make a list of the three of students, Alice, Jack, and Molly, all as strings
students = ['Alice', 'Bob', 'Carol']

# Now we just call the Series function in pandas and pass in the students
pd.Series(students)

0    Alice
1      Bob
2    Carol
dtype: object

The result is a Series object which is nicely rendered to the screen. We see here that pandas has automatically identified the type of data in this Series as "object" and set the `dytpe` parameter as appropriate. We see that the values are indexed with integers, starting at zero

We don't have to use strings. If we passed in a list of whole numbers, for instance, we could see that panda sets the type to int64. Underneath panda stores series values in a typed array using the Numpy library which we have seen before. This offers significant speedup when processing data 
versus traditional python lists.

In [3]:
# Let's create a little list of numbers
numbers = [1, 2, 3]
# And turn that into a series
pd.Series(numbers)

0    1
1    2
2    3
dtype: int64

And we see that the result is a dtype of int64 objects (depending on the device architecture you use).

How do Numpy and thus pandas handle missing data?

In Python, we have the `none` type to indicate a lack of data. But what do we do if we want to have a typed list like we do in the series object?

Underneath, pandas does some type conversion. If we create a list of strings and we have one element, a None type, pandas inserts it as a None and uses the type object for the underlying array. 

In [4]:
# Let's recreate our list of students, but leave the last one as a None
students = ['Alice', 'Bob', None]
# And let's convert this to a series
pd.Series(students)

0    Alice
1      Bob
2     None
dtype: object

__Note__: Even though the last value is missing, the slot exists in the series - it has a index value of 2.

However, if we create a list of numbers, integers or floats, and put in the None type, pandas automatically converts this to a special floating point value designated as NaN, which stands for "Not a Number".

In [5]:
# So let's create a list with a None value in it
numbers = [1, 2, None]
# And turn that into a series
pd.Series(numbers)

0    1.0
1    2.0
2    NaN
dtype: float64

You'll notice a couple of things. First, NaN is a different value. Second, pandas set the dytpe of this series to floating point numbers instead of object or ints. That might surprise you - why not just leave this as an integer? Underneath, pandas represents NaN as a floating point number, and because integers can be typecast to floats, pandas converted our integers to floats.

__Note__: Note that the slot still exists - it has an index value of 2 and it contains NaN, which can be represented as a special float value, so the overall `dtype` is not `object`.

__Tip__: If you are wondering why the list of integers you put into a Series is not floats, it's probably because some data is missing.

None and NaN are often used in the same way, to denote missing data, but pandas does not are represent them in the same way.

NaN is *NOT* equivilent to None and when we try the equality test, the result is False.

In [6]:
# Import numpy which allows us to generate an NaN value
import numpy as np
# And lets compare it to None
np.nan == None

False

It turns out that you actually can't do an equality test of NAN to itself. When you do, 
the answer is always False. 

In [7]:
np.nan == np.nan

False

Instead, you need to use special functions to test for the presence of not a number, 
such as the Numpy library isnan().

In [8]:
np.isnan(np.nan)

True

So NaN has a similar meaning to None, but it's a numeric value and treated differently for efficiency reasons.

Creating a series from a list is common but often you have label data that you want to manipulate. A series can be created directly from dictionary data. If you do this, the index is automatically assigned to the keys of the dictionary that you provided and not just auto-incrementing integers.

Here's an example using some data of students and their classes.

In [9]:
students_scores = {'Alice': 'Programming',
                   'Bob': 'Cryptography',
                   'Carol': 'Networking'}
s = pd.Series(students_scores)
s

Alice     Programming
Bob      Cryptography
Carol      Networking
dtype: object

We see that, since it was string data, pandas set the data type of the series to "object".
We see that the index, the first column, is also a list of strings.

Once the series has been created, we can get the index object using the index attribute.

In [10]:
s.index

Index(['Alice', 'Bob', 'Carol'], dtype='object')

Notice that a lot of things are implemented as numpy
arrays, and have the `dtype` value set. This is true of indexes, and here pandas infered that we were using objects for the index.

The dtype of object is not just for strings, but for
arbitrary objects. What about a more complex type of data - say, a a list of tuples/

In [11]:
students = [("Alice","Brown"), ("Bob", "White"), ("Carol", "Green")]
pd.Series(students)

0    (Alice, Brown)
1      (Bob, White)
2    (Carol, Green)
dtype: object

We see that each of the tuples is stored in the series object, and the type is object.

You can also separate your index creation from the data by passing in the index as a list explicitly to the series.

In [12]:
s = pd.Series(['Programming', 'Cryptography', 'Networking'], index=['Alice', 'Bob', 'Carol'])
s

Alice     Programming
Bob      Cryptography
Carol      Networking
dtype: object

So what happens if your list of values in the index object are not aligned with the keys in your dictionary for creating the series?

Pandas will ignore from your dictionary all keys which are not in your index, and pandas will add None or NaN type values for any index value you provide, which is not in your dictionary key list.

Here's an example - given a dictionary of three items, in this case students and their courses

In [13]:
students_scores = {'Alice': 'Programming',
                   'Bob': 'Cryptography',
                   'Carol': 'Netowrking'}
# When I create the series object though I'll only ask for an index with three students, and
# We exclude Bob and add Dave
s = pd.Series(students_scores, index=['Alice', 'Carol', 'Dave'])
s

Alice    Programming
Carol     Netowrking
Dave             NaN
dtype: object

The result is that the Series object doesn't have Bob in it, even though he was in our original dataset, but it explicitly does have Dave in it as a missing value.

__Summary__ We have explored the pandas `Series` data structure and seen how to create a series from lists and dictionaries, how indicies on data work, and the way that pandas typecasts data including missing values.